# Biogen - List

In [84]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *
#import consts

## Import PDF

In [85]:
# Read pdf into DataFrame
df = tabula.read_pdf("EFPIA_2017_Disclosure_Report_Switzerland.pdf", pages='all', lattice=True)

## Format Table

In [86]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty', 'name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'total', 'empty1', 'empty2']

#Remove "HCP" & "HCO"
df_export.loc[df_export['empty'].str.contains("HCP", na=False), 'empty'] = np.NaN
df_export.loc[df_export['empty'].str.contains("HCO", na=False), 'empty'] = np.NaN
df_export.loc[df_export['empty'].str.contains("R&D", na=False), 'empty'] = np.NaN

#Shift all where empty not empty
df_export[df_export['empty'].notna()] = df_export[df_export['empty'].notna()].shift(1, axis='columns')

#Copy empty1 to total, if not empty
df_export.loc[df_export['empty1'].notna(), 'total'] = df_export['empty1']

#Remove rows which have no values
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.reset_index(drop=True)

#Remove empty
df_export.drop(columns=['empty', 'empty1', 'empty2'], inplace=True)

#Convert to Numbers
df_export = cleanup_number(df_export)
df_export = remove_comma(df_export)
df_export = amounts_to_number(df_export)

#Set Type
df_export = set_type_by_alphabetical_order(df_export)


#Remove Carrination
df_export = remove_carination(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

#Add PLZ
df_export = add_plz(df_export)

#Export
export_list(df_export, 'biogen')


Total nicht Summe der Werte
saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx')